# Vitessce Widget Tutorial

# Visualization of ComparativeData object

## Configure Vitessce

In [1]:
from oxc_py import transform
from vitessce import VitessceConfig, hconcat, vconcat

## Configure the data and views

In [2]:
# Reference: https://github.com/vitessce/vitessce/blob/main/examples/configs/src/view-configs/kpmp-premiere.js

In [3]:
base_url = 'https://storage.googleapis.com/vitessce-demo-data/kpmp-jan-2025/kpmp_premiere_20250330.adata.zarr'

In [4]:
vc = VitessceConfig(schema_version="1.0.17", name='Lake et al.')

dataset = vc.add_dataset('lake_et_al').add_file(
    file_type='comparisonMetadata.anndata.zarr',
    url=base_url,
    options={
      "path": 'uns/comparison_metadata',
    },
    coordination_values={
      "obsType": 'cell',
      "sampleType": 'sample',
    },
).add_file(
    file_type='comparativeFeatureStats.anndata.zarr',
    url=base_url,
    options= {
      "metadataPath": 'uns/comparison_metadata',
      "indexColumn": 'names',
      "pValueColumn": 'pvals_adj',
      "foldChangeColumn": 'logfoldchanges',
      "pValueAdjusted": True,
      "foldChangeTransformation": 'log2',
    },
    coordination_values={
      "obsType": 'cell',
      "sampleType": 'sample',
      "featureType": 'gene',
    },
).add_file(
    file_type= 'comparativeObsSetStats.anndata.zarr',
    url= base_url,
    options= {
      "metadataPath": 'uns/comparison_metadata',
      "indexColumn": 'Cell Type',
      "interceptExpectedSampleColumn": 'Expected Sample_intercept',
      "effectExpectedSampleColumn": 'Expected Sample_effect',
      "foldChangeColumn": 'log2-fold change',
      "foldChangeTransformation": 'log2',
      "isCredibleEffectColumn": 'is_credible_effect',
    },
    coordination_values= {
      "obsType": 'cell',
      "sampleType": 'sample',
    },
).add_file(
  file_type='comparativeFeatureSetStats.anndata.zarr',
  url=base_url,
  options= {
    "metadataPath": 'uns/comparison_metadata',
    "indexColumn": 'pathway_name',
    "termColumn": 'pathway_term',
    "pValueColumn": 'pvals_adj',
    "pValueAdjusted": True,
    "analysisType": 'pertpy_hypergeometric',
    "featureSetLibrary": 'Reactome_2022',
  },
  coordination_values= {
    "obsType": 'cell',
    "featureType": 'gene',
    "sampleType": 'sample',
  },
).add_file(
  file_type='anndata.zarr',
  url=base_url,
  coordination_values={
    "obsType": 'cell',
    "featureType": 'gene',
    "featureValueType": 'expression',
    "sampleType": 'sample',
  },
  options={
    "obsFeatureMatrix": {
      "path": 'layers/pearson_residuals',
    },
    "obsEmbedding": [
      {
        "path": 'obsm/X_densmap',
        "embeddingType": 'densMAP',
      },
    ],
    "obsSets": [
      {
        "name": 'Cell Type',
        "path": 'obs/cell_type',
      },
      {
        "name": 'Subclass L1',
        "path": 'obs/subclass_l1',
      },
      {
        "name": 'Subclass L2',
        "path": 'obs/subclass_l2',
      },
      {
        "name": 'Donor ID',
        "path": 'obs/donor_id',
      },
    ],
    "sampleEdges": {
      "path": 'obs/SampleID',
    },
  },
).add_file(
  file_type='sampleSets.anndata.zarr',
  url=f"{base_url}/uns/__all__.samples",
  options={
    "sampleSets": [
      {
        "name": 'Disease Type',
        "path": 'diseasetype',
      },
      {
        "name": 'Adjudicated Category',
        "path": 'AdjudicatedCategory',
      },
      {
        "name": 'Enrollment Category',
        "path": 'EnrollmentCategory',
      },
    ],
  },
  coordination_values= {
    "sampleType": 'sample',
  },
)

biomarkerSelect = vc.add_view('biomarkerSelect', dataset=dataset, uid='biomarker-select')
comparativeHeading = vc.add_view('comparativeHeading', dataset=dataset, uid='comparative-heading')
dualScatterplot = vc.add_view('dualScatterplot', dataset=dataset, uid='scatterplot')
obsSets = vc.add_view('obsSets', dataset=dataset, uid='cell-sets')
sampleSets = vc.add_view('sampleSetPairManager', dataset=dataset, uid='sample-sets')
obsSetSizes = vc.add_view('obsSetSizes', dataset=dataset)
featureList = vc.add_view('featureList', dataset=dataset)
violinPlots = vc.add_view('obsSetFeatureValueDistribution', dataset=dataset, uid='violin-plot')
dotPlot = vc.add_view('dotPlot', dataset=dataset, uid='dot-plot')
treemap = vc.add_view('treemap', dataset=dataset, uid='treemap')
volcanoPlot = vc.add_view('volcanoPlot', dataset=dataset, uid='volcano-plot')
volcanoPlotTable = vc.add_view('featureStatsTable', dataset=dataset, uid='volcano-plot-table')
obsSetCompositionBarPlot = vc.add_view('obsSetCompositionBarPlot', dataset=dataset, uid='sccoda-plot')
featureSetEnrichmentBarPlot = vc.add_view('featureSetEnrichmentBarPlot', dataset=dataset, uid='pathways-plot')

[sampleSetScope_caseControl] = vc.add_coordination('sampleSetSelection')
sampleSetScope_caseControl.set_value([['Disease Type', 'CKD'], ['Disease Type', 'Reference']])

[featureSelectionScope] = vc.add_coordination('featureSelection')
featureSelectionScope.set_value(['UMOD', 'NPHS2'])

vc.link_views_by_dict([dualScatterplot], {
    "embeddingType": 'densMAP',
    "embeddingContoursVisible": True,
    "embeddingPointsVisible": False,
    "embeddingObsSetLabelsVisible": True,
}, meta=False);


vc.link_views([biomarkerSelect, dualScatterplot, obsSets, obsSetSizes, featureList, violinPlots, dotPlot, treemap, volcanoPlot, volcanoPlotTable, comparativeHeading, obsSetCompositionBarPlot, featureSetEnrichmentBarPlot, sampleSets], ['sampleType'], ['sample'])

vc.link_views_by_dict([biomarkerSelect, dualScatterplot, obsSets, obsSetSizes, featureList, violinPlots, dotPlot, treemap, volcanoPlot, volcanoPlotTable, comparativeHeading, obsSetCompositionBarPlot, featureSetEnrichmentBarPlot, sampleSets], {
    "sampleSetSelection": sampleSetScope_caseControl,
    "featureSelection": featureSelectionScope,
}, meta=False)

vc.link_views_by_dict([dualScatterplot, violinPlots, featureList, dotPlot], {
    # "featureSelection": ['UMOD', 'NPHS2'], // , 'ENSG00000074803', 'ENSG00000164825'],
    "obsColorEncoding": 'geneSelection',
    "featureValueColormap": 'jet',
    "featureValueColormapRange": [0, 0.25],
    "featureAggregationStrategy": None,
}, meta=False)

vc.layout(hconcat(
    vconcat(dualScatterplot, biomarkerSelect, comparativeHeading, obsSets, obsSetSizes, featureList),
    vconcat(treemap, featureSetEnrichmentBarPlot, violinPlots, dotPlot, obsSetCompositionBarPlot, sampleSets),
    volcanoPlotTable,
));

In [5]:
# vc.to_dict(base_url="")

## Define the page layout

In [6]:
# Reference: https://github.com/vitessce/vitessce/blob/main/examples/configs/src/view-configs/lake-2023.js
PAGE_ESM = transform("""
function createPage(utilsForPages) {
  const {
    React,
    usePageModeView,
  } = utilsForPages;
  function PageComponent(props) {
    const BiomarkerSelect = usePageModeView('biomarker-select');
    const DualScatterplot = usePageModeView('scatterplot');
    const CellSets = usePageModeView('cell-sets');
    const ViolinPlot = usePageModeView('violin-plot');
    const DotPlot = usePageModeView('dot-plot');
    
    return (
        <>
          <style>{`
          h1, h2, h3, h4, h5, h6 {
            font-family: sans-serif;
          }
          h1 {
            font-weight: normal;
          }
          h2 {
            font-size: 36px;
          }
          h3 {
            font-size: 28px;
          }
          `}
          </style>
          <div style={{ width: '100%' }}>
              <h1>Comparative visualization of single-cell atlas data</h1>
              <BiomarkerSelect />
          </div>

          <div style={{ width: '100%', display: 'flex', flexDirection: 'row' }}>
            <div style={{ width: '100%'}}>
              <div style={{ width: '100%', display: 'flex', flexDirection: 'row' }}>
                <div style={{ width: '45%' }}><h2>Chronic Kidney Disease</h2></div>
                <div style={{ width: '5%' }}><h2 style={{ textAlign: 'right' }}>vs.&nbsp;</h2></div>
                <div style={{ width: '50%' }}><h2>Healthy Reference</h2></div>
              </div>
              <h3>Cell type-level representations</h3>
              <div style={{ width: '100%', height: '500px' }}>
                <DualScatterplot />
              </div>
              <div style={{ width: '100%', height: '500px' }}>
                <ViolinPlot />
              </div>
              <div style={{ width: '100%', height: '500px' }}>
                <DotPlot />
              </div>
            </div>
            <div style={{ width: '14%', height: '500px', marginTop: '213px' }}>
              <CellSets />
            </div>
          </div>

        </>
      );
  }
  return PageComponent;
}
export default { createPage };
""")

## Render page as widget

In [7]:
vw = vc.widget(page_esm=PAGE_ESM, page_mode=True, height=3000)
vw

VitessceWidget(config={'version': '1.0.17', 'name': 'Lake et al.', 'description': '', 'datasets': [{'uid': 'A'…